<a href="https://colab.research.google.com/github/srta-raimee/SimplePDF/blob/main/TDE_SimplePDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TDE 1 - SimplePDF - Construção de Interpretadores**

Aluna: Larissa Raimee Gomes

Turma: 7C

In [ ]:
import requests

config_basico = "https://raw.githubusercontent.com/srta-raimee/SimplePDF/main/basico.txt"
config_completo = "https://raw.githubusercontent.com/srta-raimee/SimplePDF/main/completo.txt"
config_desativado = "https://raw.githubusercontent.com/srta-raimee/SimplePDF/refs/heads/main/desativado.txt"
teste1_link = "https://raw.githubusercontent.com/srta-raimee/SimplePDF/main/teste1.txt"
teste2_link = "https://raw.githubusercontent.com/srta-raimee/SimplePDF/main/teste2.txt"
teste3_link = "https://raw.githubusercontent.com/srta-raimee/SimplePDF/main/teste3.txt"

# requests pro github e armazenando os conteúdos
response_teste1 = requests.get(teste1_link)
teste1 = response_teste1.text if response_teste1.status_code == 200 else "[ERRO] Não foi possível encontrar o arquivo de teste."

response_teste2 = requests.get(teste2_link)
teste2 = response_teste2.text if response_teste2.status_code == 200 else "[ERRO] Não foi possível encontrar o arquivo de teste."

response_teste3 = requests.get(teste3_link)
teste3 = response_teste3.text if response_teste3.status_code == 200 else "[ERRO] Não foi possível encontrar o arquivo de teste."

In [ ]:
import re
import datetime
import requests

class SimplePDF:
    def __init__(self, texto):
        self.texto_original = texto
        self.texto = self._remover_comentarios(texto)
        self.erros = []                             # lista para armazenar todos os erros
        self.count = 0                              # quantidade de entradas no xref
        self.objetos_info = []                      # infos de cada objeto
        self.objetos_raw = self._extrair_objetos()  # cache de objetos
        self.ciclos = []                            # detecçao de ciclos

    @staticmethod
    def _remover_comentarios(texto):
      """removendo todas as linhas com % exceto o cabeçalho"""
      linhas = texto.splitlines()
      if not linhas:
          return texto
      texto_filtrado = [linhas[0]]
      for linha in linhas[1:]:
          strip = linha.strip()
          if strip.startswith("%%EOF") or not strip.startswith("%"):
              texto_filtrado.append(linha)
      return "\n".join(texto_filtrado)

    @staticmethod
    def _get_obj_base_type(conteudo):
        """determina o tipo base do objeto a partir do conteúdo."""
        if '/Type /Catalog' in conteudo:
            return 'Catalog'
        elif '/Type /Pages' in conteudo:
            return 'Pages'
        elif '/Type /Page' in conteudo:
            return 'Page'
        elif '/Type /Font' in conteudo or '/Type /FontDescriptor' in conteudo:
            return 'Font'
        elif '/Type /Outlines' in conteudo:
            return 'Outlines'
        elif 'stream' in conteudo and 'endstream' in conteudo:
            return 'Contents (stream)'
        else:
            return 'Metadata'

    @staticmethod
    def _get_obj_label(conteudo):
        """retorna o rótulo do objeto para validação."""
        if re.search(r'/Type\s*/Catalog', conteudo):
            return "Catálogo (root)"
        elif re.search(r'/Type\s*/Pages', conteudo):
            return "Objeto de Páginas (lista de páginas)"
        elif re.search(r'/Type\s*/Page', conteudo) or re.search(r'/Type\s*/Font', conteudo):
            return "Objeto de Página (conteúdo e propriedades)"
        elif re.search(r'stream.*?endstream', conteudo, re.DOTALL):
            return "Objeto de Conteúdo (texto e elementos gráficos)"
        else:
            return "Objeto de Metadados"

    def _extrair_objetos(self):
        '''pegando cada objeto do arquivo'''
        return re.findall(r'\d+\s+\d+\s+obj.*?endobj', self.texto, re.DOTALL)

    def _construir_obj_dict(self):
        objetos = re.findall(r'(\d+)\s+\d+\s+obj(.*?)endobj', self.texto, re.DOTALL)
        if not objetos:
            print("[ERRO] Nenhum objeto encontrado.")
            return {}
        obj_dict = {}
        for num_str, conteudo in objetos:
            num = int(num_str)
            tipo = self._get_obj_base_type(conteudo)
            refs = [int(r) for r in re.findall(r'(\d+)\s+\d+\s+R', conteudo)]
            obj_dict[num] = {'type': tipo, 'refs': refs, 'content': conteudo}
        return obj_dict

    def validar_cabecalho(self):
        '''validando o cabeçalho se for igual ao esperado'''
        if re.match(r'^%SPDF-1\.\d', self.texto_original):
            print("[OK] Cabeçalho válido")
            return True
        else:
            msg = "[ERRO] Cabeçalho inválido"
            print(msg)
            self.erros.append("[CABEÇALHO] Erro na sintaxe do cabeçalho ou cabeçalho não encontrado.")
            return False

    def validar_objetos(self):
        '''validando a estrutura dos objetos e tratando erros'''
        objetos = self.objetos_raw
        self.objetos_info = []
        erros_objetos = []
        for idx, obj in enumerate(objetos, 1):
            if "<<" not in obj or ">>" not in obj:
                erros_objetos.append(f"[OBJETO] O dicionário do objeto {idx} não foi definido corretamente")
                continue
            if "stream" in obj and "endstream" not in obj:
                erros_objetos.append(f"[STREAM] Objeto {idx} contém stream sem endstream")
                continue
            obj_info = {"Objeto": idx, "Tipo": self._get_obj_label(obj)}
            self.objetos_info.append(obj_info)
        if erros_objetos:
            print("[ERRO] Sintaxe de objetos")
            self.erros.extend(erros_objetos)
        else:
            print("[OK] Sintaxe de objetos")

    def validar_trailer(self):
        ''' validando o trailer do documento comparando ele com o padrão'''
        trailer_pattern = re.compile(
            r'trailer\s*<<\s*'
            r'/Size\s+(\d+)\s*'
            r'/Root\s+(\d+)\s+\d+\s+R\s*'
            r'(?:/.*?\s*)*'
            r'>>\s*'
            r'startxref\s*(\d+)\s*'
            r'%%EOF\s*$',
            re.DOTALL
        )
        if trailer_pattern.search(self.texto):
            print("[OK] Trailer válido")
            return True
        else:
            msg = "[ERRO] Trailer inválido"
            print(msg)
            self.erros.append("[TRAILER] Erro na sintaxe do trailer ou trailer não encontrado.")
            return False

    def arvore_obj(self):
        '''algoritmo de busca em profundidade pra montar a árvore de objetos com base nas referências'''
        obj_dict = self._construir_obj_dict()
        if not obj_dict:
            return

        ciclos_reportados = set()
        def detect_cycle(obj_num, path):
            if obj_num in path:
                if obj_num not in ciclos_reportados:
                    self.ciclos.append(f"[CICLO] Ciclo de referência encontrado no objeto {obj_num}")
                    ciclos_reportados.add(obj_num)
                return
            path.add(obj_num)
            for ref in obj_dict[obj_num]['refs']:
                if ref in obj_dict:
                    detect_cycle(ref, path.copy())
        for num in obj_dict.keys():
            detect_cycle(num, set())

        visited = set()
        def print_tree(obj_num, indent=""):
            if obj_num in visited:
                return
            visited.add(obj_num)
            print(f"{indent}{obj_num}: {obj_dict[obj_num]['type']}")
            for ref in obj_dict[obj_num]['refs']:
                if ref in obj_dict:
                    print_tree(ref, indent + " +- ")
        raiz = next((num for num, info in obj_dict.items() if info['type'] == 'Catalog'), None)
        if raiz is not None:
            print_tree(raiz)
            all_visited = set()
            def coletar(obj_num):
                if obj_num in all_visited:
                    return
                all_visited.add(obj_num)
                for ref in obj_dict[obj_num]['refs']:
                    if ref in obj_dict:
                        coletar(ref)
            coletar(raiz)
            for num in sorted(obj_dict.keys()):
                if num not in all_visited:
                    print(f"{num}: {obj_dict[num]['type']}")
        else:
            for num in sorted(obj_dict.keys()):
                print(f"{num}: {obj_dict[num]['type']}")

    def validar_xref(self):
      '''validando o xref com base no padrão'''
      pattern = re.compile(
          r"xref\s+"
          r"(\d+)\s+(\d+)\s*\n"
          r"((?:\d{10}\s\d{5}\s[fn]\s*\n?)+)",
          re.DOTALL
      )
      match = pattern.search(self.texto)
      if not match:
          msg = "[ERRO] XREF inválido"
          print(msg)
          self.erros.append("[XREF] Erro na sintaxe do xref ou xref não encontrado.")
          return False
      start_obj = int(match.group(1))
      self.count = int(match.group(2))
      entries = [linha.strip() for linha in match.group(3).strip().splitlines() if linha.strip()]
      if len(entries) != self.count:
          msg = "[XREF] inválido: número de entradas não corresponde ao header"
          print("[ERRO] XREF inválido")
          self.erros.append(msg)
          return False
      for idx, entry in enumerate(entries, start=start_obj):
          if not re.fullmatch(r"\d{10}\s\d{5}\s[fn]", entry):
              msg = f"XREF inválido: entrada '{entry}' no índice {idx} com formato incorreto"
              self.erros.append(msg)
              return False
          offset = int(entry.split()[0])
          if offset >= len(self.texto):
              msg = f"XREF inválido: offset {offset} na entrada {idx} excede tamanho do documento"
              self.erros.append(msg)
              return False
      print("[OK] XREF válido")
      return True

    def validar_referencias(self):
        '''validando as referências de cada um dos objetos'''
        valid_ids = {int(re.match(r'(\d+)', obj).group(1)) for obj in self.objetos_raw if re.match(r'(\d+)', obj)}
        erros_referencias = []
        for obj in self.objetos_raw:
            refs = re.findall(r'(\d+)\s+\d+\s+R', obj)
            for ref in refs:
                if int(ref) not in valid_ids:
                    erro_msg = f"[REFERENCIAS] Referência inválida: objeto {ref} não encontrado."
                    if erro_msg not in erros_referencias:
                        erros_referencias.append(erro_msg)
        if erros_referencias:
            print("[ERRO] Referências")
            self.erros.extend(erros_referencias)
        else:
            print("[OK] Referências")
        return not erros_referencias

    def extrair_infos(self, nivel_detalhe=None):
        '''extraindo informações importantes do simplepdf'''
        def extrair_campo(campo, default="Não encontrado"):
            match = re.search(rf'/{campo}\s*\((.*?)\)', self.texto)
            return match.group(1) if match else default

        titulo = extrair_campo("Title")
        autor = extrair_campo("Author")
        data_raw = extrair_campo("CreationDate")
        if data_raw != "Não encontrado" and data_raw.startswith("D:"):
            data_raw = data_raw[2:]
            try:
                dt = datetime.datetime.strptime(data_raw, "%Y%m%d%H%M%S")
                data = dt.strftime("%d-%m-%Y %H:%M:%S")
            except Exception:
                data = data_raw
        else:
            data = data_raw

        streams = re.findall(r'stream(.*?)endstream', self.texto, re.DOTALL)
        conteudos = []
        for s in streams:
            conteudos.extend(re.findall(r'\((.*?)\)', s))
        texto_extraido2 = "\n".join(conteudos)
        texto_extraido = texto_extraido2 if len(texto_extraido2) <= 200 else texto_extraido2[:200] + "..."
        print(f"Título: {titulo}")
        print(f"Autor: {autor}")
        print(f"Data de criação: {data}")
        print("\n-- Texto extraído --")
        print(texto_extraido)
        with open("extraido.txt", "w", encoding="utf-8") as f:
            f.write(texto_extraido2)

    def coordenadas(self):
        '''convertendo as coordenadas de texto para algo mais legível'''
        obj_dict = self._construir_obj_dict()
        m = re.search(r'/MediaBox\s*\[\s*\d+(?:\.\d+)?\s+\d+(?:\.\d+)?\s+([\d.]+)\s+([\d.]+)', self.texto)
        default_page_height = float(m.group(2)) if m else 792
        for obj_num, info in obj_dict.items():
            if info['type'] == 'Contents (stream)':
                tds = re.findall(r'(-?\d+(?:\.\d+)?)\s+(-?\d+(?:\.\d+)?)\s+Td', info['content'])
                current_x, current_y = None, None
                for x_str, y_str in tds:
                    x = float(x_str)
                    y = float(y_str)
                    if current_x is None:
                        current_x, current_y = x, y
                    else:
                        current_x += x
                        current_y += y
                    top = default_page_height - current_y
                    print(f"Objeto {obj_num}: Posição: {current_x}px da esquerda, {top}px do topo da página")

    def estatisticas(self):
      '''extraindo estatísticas do documento'''
      total_objetos = len(self.objetos_raw)
      tipos = {"Catalog": 0, "Pages": 0, "Page": 0, "Font": 0, "Conteudo": 0, "Metadados": 0, "Outlines": 0}
      self.fontes_utilizadas = set()

      for obj in self.objetos_raw:
          base_type = self._get_obj_base_type(obj)
          if base_type == "Catalog":
              tipos["Catalog"] += 1
          elif base_type == "Pages":
              tipos["Pages"] += 1
          elif base_type == "Page":
              tipos["Page"] += 1
          elif base_type == "Font":
              tipos["Font"] += 1
              match = re.search(r'/BaseFont\s*/([A-Za-z0-9#\-]+)', obj)
              if match:
                  self.fontes_utilizadas.add(match.group(1))
          elif base_type == "Contents (stream)":
              tipos["Conteudo"] += 1
          elif base_type == "Outlines":
              tipos["Outlines"] += 1
          else:
              tipos["Metadados"] += 1

      total_paginas = tipos["Page"]
      tamanho_documento = len(self.texto.encode('utf-8'))
      streams = re.findall(r'stream(.*?)endstream', self.texto, re.DOTALL)
      total_stream_bytes = sum(len(s.strip('\r\n').encode('utf-8')) for s in streams)
      overhead_bytes = tamanho_documento - total_stream_bytes
      overhead_percent = (overhead_bytes / tamanho_documento * 100) if tamanho_documento > 0 else 0

      print(f"Total de entradas (XREF): {self.count}")
      print(f"Total de objetos: {total_objetos}")
      tipos_str = []
      for key in ["Catalog", "Pages", "Page", "Font", "Conteudo", "Metadados", "Outlines"]:
          if tipos[key]:
              tipos_str.append(f"{key}={tipos[key]}")
      print("Objetos por tipo: " + ", ".join(tipos_str))
      print(f"Total de páginas: {total_paginas}")
      print(f"Tamanho do documento: {tamanho_documento} bytes")
      print(f"Overhead estrutural: {overhead_bytes} bytes ({overhead_percent:.2f}%)")


    def cria_sumario(self):
        '''criando um sumário do conteúdo do pdf'''
        obj_dict = self._construir_obj_dict()
        if not obj_dict:
            print("Nenhum outline encontrado.")
            return
        outlines_root = next((num for num, info in obj_dict.items() if info['type'] == 'Outlines'), None)
        if outlines_root is None:
            print("Nenhum outline encontrado.")
            return
        first_match = re.search(r'/First\s+(\d+)\s+\d+\s+R', obj_dict[outlines_root]['content'])
        if not first_match:
            print("Nenhum outline encontrado.")
            return
        first_outline = int(first_match.group(1))
        def traverse(item_num, level=0):
            if item_num not in obj_dict:
                return []
            content = obj_dict[item_num]['content']
            title_match = re.search(r'/Title\s*\((.*?)\)', content)
            title = title_match.group(1) if title_match else "Sem Título"
            lines = [("  " * level) + "- " + title]
            first_child = re.search(r'/First\s+(\d+)\s+\d+\s+R', content)
            if first_child:
                child_num = int(first_child.group(1))
                lines.extend(traverse(child_num, level+1))
            next_item = re.search(r'/Next\s+(\d+)\s+\d+\s+R', content)
            if next_item:
                next_num = int(next_item.group(1))
                lines.extend(traverse(next_num, level))
            return lines
        summary = traverse(first_outline)
        print("\n".join(summary))

    @staticmethod
    def processa_arquivo_config(config_file):
        '''processando o arquivo de configuração para setar as opções com arquivo de config'''
        opcoes = {
            'extrair_texto': True,
            'gerar_sumario': True,
            'detectar_ciclos': True,
            'nivel_detalhe': 'basico',
            'validar_xref': True
        }
        if config_file.lower().startswith("http"):
            try:
                r = requests.get(config_file)
                r.raise_for_status()
                conteudo = r.text.splitlines()
            except Exception as e:
                print(f"[ERRO] Não foi possível baixar o arquivo de config: {e}")
                return opcoes
        else:
            with open(config_file, 'r', encoding='utf-8') as f:
                conteudo = f.readlines()

        for linha in conteudo:
            linha = linha.strip()
            if not linha or '=' not in linha:
                continue
            chave, valor = map(str.strip, linha.split('=', 1))
            valor = valor.lower()
            if chave == 'extrair_texto':
                opcoes['extrair_texto'] = (valor == 'sim')
            elif chave == 'gerar_sumario':
                opcoes['gerar_sumario'] = (valor == 'sim')
            elif chave == 'detectar_ciclos':
                opcoes['detectar_ciclos'] = (valor == 'sim')
            elif chave == 'nivel_detalhe':
                opcoes['nivel_detalhe'] = valor
            elif chave == 'validar_xref':
                opcoes['validar_xref'] = (valor == 'sim')
        return opcoes

    @classmethod
    def _processar_texto(cls, texto, opcoes):
        if not texto.strip():
            print("[ERRO] O documento está vazio")
            return
        if not re.search(r'\d+\s+\d+\s+obj', texto):
            print("[ERRO] Nenhum objeto encontrado. Interrompendo processamento.")
            return
        pdf = cls(texto)
        print("\n--- Validação ---")
        pdf.validar_cabecalho()
        pdf.validar_objetos()
        pdf.validar_trailer()
        if opcoes['validar_xref']:
            pdf.validar_xref()
        else:
            print("[INFO] Validação de xref desativada (config)")
        pdf.validar_referencias()
        print("\n--- Estatísticas ---")
        pdf.estatisticas()
        if opcoes['detectar_ciclos']:
            print("\n--- Árvore de Objetos ---")
            pdf.arvore_obj()
        else:
            print("\n[INFO] Detecção de ciclos (árvore) desativada (config)")
        if opcoes['gerar_sumario']:
            print("\n--- Sumário ---")
            pdf.cria_sumario()
        else:
            print("\n[INFO] Geração de sumário desativada (config)")
        if opcoes['extrair_texto']:
            print("\n--- Conteúdo Extraído ---")
            pdf.extrair_infos(nivel_detalhe=opcoes['nivel_detalhe'])
        else:
            print("\n[INFO] Extração de texto desativada (config)")

        if opcoes['nivel_detalhe'] == 'completo':
            print("\n--- Fontes Utilizadas ---")
            if pdf.fontes_utilizadas:
                for fonte in pdf.fontes_utilizadas:
                    print(f"- {fonte}")
            else:
                print("Nenhuma fonte encontrada.")
            if pdf.erros:
                print("\n--- Detalhamento de Erros ---")
                for erro in pdf.erros:
                    print("-", erro)
            print("\n--- Coordenadas de texto convertidas ---")
            pdf.coordenadas()

            if pdf.ciclos:
                print("\n--- Ciclos de referência detectados ---")
                for c in pdf.ciclos:
                    print(c)

        else:
            print("\n[INFO] Modo básico: serão exibidas apenas as informações essenciais.")
        if not pdf.erros:
            print("\nNenhum erro encontrado.")

    @classmethod
    def processar_simplepdf_local(cls, nome_arquivo, config_file=None):
        opcoes = {
            'extrair_texto': True,
            'gerar_sumario': True,
            'detectar_ciclos': True,
            'nivel_detalhe': 'completo',
            'validar_xref': True
        }
        if config_file:
            config_opcoes = cls.processa_arquivo_config(config_file)
            opcoes.update(config_opcoes)
        with open(nome_arquivo, 'r', encoding='utf-8') as arquivo:
            texto = arquivo.read()
        cls._processar_texto(texto, opcoes)

    @classmethod
    def processar_simplepdf(cls, texto, config_file=None):
        opcoes = {
            'extrair_texto': True,
            'gerar_sumario': True,
            'detectar_ciclos': True,
            'nivel_detalhe': 'completo',
            'validar_xref': True
        }
        if config_file:
            config_opcoes = cls.processa_arquivo_config(config_file)
            opcoes.update(config_opcoes)
        cls._processar_texto(texto, opcoes)


In [ ]:
# Arquivo disponibilizado pelo professor no pdf do trabalho
SimplePDF.processar_simplepdf(teste1, config_file=config_completo)


--- Validação ---
[OK] Cabeçalho válido
[OK] Sintaxe de objetos
[OK] Trailer válido
[OK] XREF válido
[OK] Referências

--- Estatísticas ---
Total de entradas (XREF): 15
Total de objetos: 14
Objetos por tipo: Catalog=1, Pages=1, Page=2, Font=2, Conteudo=3, Metadados=4, Outlines=1
Total de páginas: 2
Tamanho do documento: 2811 bytes
Overhead estrutural: 1936 bytes (68.87%)

--- Árvore de Objetos ---
1: Catalog
 +- 2: Pages
 +-  +- 3: Page
 +-  +-  +- 5: Font
 +-  +-  +-  +- 10: Font
 +-  +-  +- 6: Contents (stream)
 +-  +- 4: Page
 +-  +-  +- 9: Contents (stream)
 +- 7: Metadata
 +- 8: Outlines
 +-  +- 11: Metadata
 +-  +-  +- 12: Metadata
 +-  +-  +-  +- 13: Metadata
14: Contents (stream)

--- Sumário ---
- Resumo Executivo
- Detalhamento de Vendas
- Projeções Futuras

--- Conteúdo Extraído ---
Título: Relatório Financeiro Trimestral
Autor: Departamento Financeiro
Data de criação: 18-04-2023 09:00:00

-- Texto extraído --
Relatório Financeiro: Primeiro Trimestre
Total de vendas: 1423 u

In [ ]:
# Arquivo de exemplo com 21 objetos, alguns erros e configuração completa com todos os demais itens desativados
SimplePDF.processar_simplepdf(teste2, config_file=config_desativado)


--- Validação ---
[ERRO] Cabeçalho inválido
[ERRO] Sintaxe de objetos
[ERRO] Trailer inválido
[INFO] Validação de xref desativada (config)
[ERRO] Referências

--- Estatísticas ---
Total de entradas (XREF): 0
Total de objetos: 21
Objetos por tipo: Catalog=1, Pages=1, Page=5, Font=2, Conteudo=7, Metadados=4, Outlines=1
Total de páginas: 5
Tamanho do documento: 2616 bytes
Overhead estrutural: 2057 bytes (78.63%)

[INFO] Detecção de ciclos (árvore) desativada (config)

[INFO] Geração de sumário desativada (config)

[INFO] Extração de texto desativada (config)

--- Fontes Utilizadas ---
- Helvetica

--- Detalhamento de Erros ---
- [CABEÇALHO] Erro na sintaxe do cabeçalho ou cabeçalho não encontrado.
- [OBJETO] O dicionário do objeto 9 não foi definido corretamente
- [TRAILER] Erro na sintaxe do trailer ou trailer não encontrado.
- [REFERENCIAS] Referência inválida: objeto 100 não encontrado.

--- Coordenadas de texto convertidas ---
Objeto 5: Posição: 50.0px da esquerda, 42.0px do topo da 

In [ ]:
# Arquivo de exemplo com 40 objetos
SimplePDF.processar_simplepdf(teste3, config_file=config_completo)


--- Validação ---
[OK] Cabeçalho válido
[OK] Sintaxe de objetos
[OK] Trailer válido
[OK] XREF válido
[OK] Referências

--- Estatísticas ---
Total de entradas (XREF): 41
Total de objetos: 40
Objetos por tipo: Catalog=1, Pages=1, Page=10, Font=2, Conteudo=11, Metadados=14, Outlines=1
Total de páginas: 10
Tamanho do documento: 5136 bytes
Overhead estrutural: 3439 bytes (66.96%)

--- Árvore de Objetos ---
1: Catalog
 +- 2: Pages
 +-  +- 4: Page
 +-  +-  +- 5: Contents (stream)
 +-  +- 6: Page
 +-  +-  +- 7: Contents (stream)
 +-  +- 8: Page
 +-  +-  +- 9: Contents (stream)
 +-  +- 10: Page
 +-  +-  +- 11: Contents (stream)
 +-  +- 12: Page
 +-  +-  +- 13: Contents (stream)
 +-  +- 14: Page
 +-  +-  +- 15: Contents (stream)
 +-  +- 16: Page
 +-  +-  +- 17: Contents (stream)
 +-  +- 18: Page
 +-  +-  +- 19: Contents (stream)
 +-  +- 20: Page
 +-  +-  +- 21: Contents (stream)
 +-  +- 22: Page
 +-  +-  +- 23: Contents (stream)
 +- 3: Outlines
 +-  +- 31: Metadata
 +-  +-  +- 32: Metadata
 +- 